In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.mixed_precision import experimental as mixed_precision


In [2]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)


Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  GeForce GTX 1080 Ti, compute capability 6.1
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale


In [13]:
img_width, img_height = 100, 100

train_data_dir = 'data/img/trainData'
val_data_dir = 'data/img/valData'
test_data_dir = 'data/img/testData'


In [14]:
import os 
print(sum([len(os.listdir(os.path.join(train_data_dir, path))) for path in os.listdir(train_data_dir)]))
print(sum([len(os.listdir(os.path.join(val_data_dir, path))) for path in os.listdir(val_data_dir)]))
print(sum([len(os.listdir(test_data_dir))]))

89992
10009
20000


In [15]:

nb_train_samples = 89992
nb_validation_samples = 10009

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

## data

In [16]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=False)

In [17]:
test_datagen = ImageDataGenerator(rescale=1. / 255)


In [18]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 89992 images belonging to 4 classes.


In [19]:
validation_generator = test_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 10009 images belonging to 4 classes.


## model

In [22]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax',dtype='float32'))

# outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
activation_5 (Activation)    (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
activation_6 (Activation)    (None, 47, 47, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 21, 21, 64)       

In [25]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


  opt = tf.keras.mixed_precision.LossScaleOptimizer(opt)



In [27]:
epochs = 20
batch_size = 64

In [28]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

/environment/python/versions/miniconda3-4.7.12/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
1406/1406 [==============================] - 106s 76ms/step - loss: 0.2160 - accuracy: 0.9335 - val_loss: 0.1366 - val_accuracy: 0.9551
Epoch 2/20
1406/1406 [==============================] - 106s 76ms/step - loss: 0.1792 - accuracy: 0.9408 - val_loss: 0.1179 - val_accuracy: 0.9655
Epoch 3/20
1406/1406 [==============================] - 106s 76ms/step - loss: 0.1638 - accuracy: 0.9502 - val_loss: 0.1047 - val_accuracy: 0.9655
Epoch 4/20
1406/1406 [==============================] - 106s 76ms/step - loss: 0.1518 - accuracy: 0.9532 - val_loss: 0.0941 - val_accuracy: 0.9700
Epoch 5/20
1406/1406 [==============================] - 106s 76ms/step - loss: 0.1384 - accuracy: 0.9580 - val_loss: 0.0855 - val_accuracy: 0.9708
Epoch 6/20
1406/1406 [==============================] - 106s 76ms/step - loss: 0.1303 - accuracy: 0.9580 - val_loss: 0.0708 - val_accuracy: 0.9792
Epoch 7/20
1406/1406 [==============================] - 106s 75ms/step - loss: 0.1183 - accuracy: 0.9614 - val_loss: 0